# Works on CIT where posteriors are available

In [1]:
import  numpy as np, os, sys, glob, json, h5py, argparse
import seaborn as sns
from pesummary.io import read
#from pesummary.gw.file.read import read
import matplotlib.pylab as plt
import pandas as pd

In [2]:
#base_dir = '/home1/srashti.goyal/strong-lensing-ml-backup-8aug21/' #Alice
base_dir = '/home/srashti.goyal/strong-lensing-ml/' #CIT

In [3]:
O3a_events_df=pd.read_csv(base_dir + 'data/O3a_events/O3a_blu.txt',delimiter='\t')
O3a_events_df.tail()

,event1,event2,blu,rlu_haris,rlu_anupreeta
698,GW190930_133541,GW190828_065509,8.200000e-108,0.0704,0.461
699,GW190930_133541,GW190909_114149,6.460000e-82,0.1450,1.090
700,GW190930_133541,GW190910_112807,0.000000e+00,0.1560,1.190
701,GW190930_133541,GW190924_021846,0.000000e+00,0.6460,4.460
702,GW190930_133541,GW190929_012149,0.000000e+00,3.0200,9.520


In [4]:
def read_pe_samples(fname,event_name):
    data = read(fname)
    if event_name=='GW190412' or event_name=='GW190814':
        posterior_samples = data.samples_dict['C01:IMRPhenomPv3HM']
    elif event_name=='GW190521':
        posterior_samples = data.samples_dict['C01:NRSur7dq4']
    else:
        posterior_samples = data.samples_dict['C01:IMRPhenomPv2']
    return posterior_samples

In [5]:
def read_pe_samples_h5py(fname,event_name):
    data = h5py.File(fname,'r')
    if event_name=='GW190412' or event_name=='GW190814':
        posterior_samples = data['C01:IMRPhenomPv3HM']['posterior_samples']
    elif event_name=='GW190521':
        posterior_samples = data['C01:NRSur7dq4']['posterior_samples']
    else:
        posterior_samples = data['C01:IMRPhenomPv2']['posterior_samples']
    return posterior_samples

In [6]:
data_dir='/home/pe.o3/o3a_catalog/data_release/all_posterior_samples/'


In [107]:
def make_sns_corner_plots_new_params(psamples1,psamples2,fname='test.png',inj_pars1=None,inj_pars2=None,kde=False):
    """Plot and save superposed KDE corner plots for the two images from posteriors."""
    samples1=pd.DataFrame()
    samples2=pd.DataFrame()
    samples1['m1'],samples1['m2']=psamples1['mass_1'],psamples1['mass_2']
    samples1['dec']=np.sin(psamples1['ra'])
    
    samples2['m1'],samples2['m2']=psamples2['mass_1'],psamples2['mass_2']
    samples2['dec']=np.sin(psamples2['dec'])

    
    keys = ['ra','theta_jn','psi','phase','geocent_time','luminosity_distance']
    for key in keys:
        samples1[key],samples2[key] = psamples1[key],psamples2[key]
    samples1['img']=np.zeros(len(samples1['m1'])).astype(str)
    samples2['img']=np.ones(len(samples2['m1'])).astype(str)
    posteriors=pd.concat([samples1,samples2])
    sns.set(font_scale=1.5)
    posteriors=posteriors.rename(columns={"m1": "$m_1$", "m2": "$m_2$",'dec':'sin($\delta$)','ra':'$\\alpha$','theta_jn':'$\iota$','psi':'$\psi$','phase':'$\phi_0$','geocent_time': '$t_c$','luminosity_distance':'$d_L$'})
    fig1 = sns.PairGrid(data= posteriors,vars = ['$m_1$','$m_2$','$\\alpha$','sin($\delta$)','$\iota$','$\psi$','$\phi_0$','$d_L$','$t_c$'],height=1.5,corner=True,hue='img')
    if kde == True:
        fig1 = sns.PairGrid(data= posteriors,vars = ['$m_1$','$m_2$','$\\alpha$','sin($\delta$)','$\iota$','$\psi$','$\phi_0$','$d_L$','$t_c$'],height=1.5,hue='img')
        fig1 = fig1.map_lower(sns.kdeplot)#,shade=True,shade_lowest=False)

    fig1 = fig1.map_lower(sns.histplot,pthresh=0.1)
    
    fig1 = fig1.map_diag(plt.hist,histtype='step', lw=2)
    plt.subplots_adjust(hspace=0.05, wspace=0.05)
    ndim=9
    axes=fig1.axes
        
    for inj_pars in [inj_pars1,inj_pars2]:
        if inj_pars is not None:
            values = np.array([inj_pars['mass_1'],inj_pars['mass_2'],inj_pars['ra'],np.sin(inj_pars['dec']),inj_pars['theta_jn'],inj_pars['psi'],inj_pars['phase'],inj_pars['luminosity_distance'],inj_pars['geocent_time']])

            for i in range(ndim):
                ax = axes[i, i]
                ax.axvline(values[i], color="gray")
            for yi in range(ndim):
                for xi in range(yi):
                    ax = axes[yi, xi]
                    ax.axvline(values[xi], color="gray")
                    ax.axhline(values[yi], color="gray")

 
    plt.savefig(fname)
    plt.close()
    print('Done: ' + fname)

In [108]:
#!ls /home/pe.o3/o3a_catalog/data_release/all_posterior_samples/*_comoving.h5

In [109]:
odir = 'corner_plots/'

In [110]:
def plot_corners_events(i):
    posterior_file =data_dir + O3a_events_df['event1'][i]+'_comoving.h5'
    posterior_samples1 = read_pe_samples_h5py(posterior_file,O3a_events_df['event1'][i][:8])
    posterior_file =data_dir + O3a_events_df['event2'][i]+'_comoving.h5'
    posterior_samples2 = read_pe_samples_h5py(posterior_file,O3a_events_df['event2'][i][:8])
    fname=odir+O3a_events_df['event1'][i]+'-'+O3a_events_df['event2'][i]+'.png'
    make_sns_corner_plots_new_params(posterior_samples1,posterior_samples2,fname=fname)

In [111]:
i = 10
posterior_file =data_dir + O3a_events_df['event1'][i]+'_comoving.h5'

posterior_samples1 = read_pe_samples_h5py(posterior_file,O3a_events_df['event1'][i][:8])


In [112]:
plot_corners_events(10)

Done: corner_plots/GW190408_181802-GW190521_074359.png
